# Import Dataset

In [1]:
import os
import sys
import gensim
import pandas as pd
from gensim.models.doc2vec import LabeledSentence

data_physics = pd.read_csv('ad_physics.csv',index_col=0)
data_race = pd.read_csv('ad_race.csv',index_col=0)
data_religion = pd.read_csv('ad_religion.csv',index_col=0)
data_physics.head()

C:\Users\User\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


,sentence,physics,race,religion
0,I don't get why negroes always traveling to wh...,0,1,0
1,lmao how funny that true does know where the c...,0,0,0
2,@art_is_forever when did she publicly thank him?,0,0,0
3,Post a picture of Khloe already!!!!! Come on!!!!,0,0,0
4,@progreenlc no we don't. When did he become th...,0,0,0


# Preprocess Data

In [2]:
CONTRACTION_MAP = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" } 

In [3]:
import re, nltk, string
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.corpus import wordnet

stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def expand_contractions(text) :
    pattern = re.compile("({})".format("|".join(CONTRACTION_MAP.keys())),flags = re.DOTALL| re.IGNORECASE)
    
    def replace_text(t):
        txt = t.group(0)
        if txt.lower() in CONTRACTION_MAP.keys():
            return CONTRACTION_MAP[txt.lower()]
        
    expand_text = pattern.sub(replace_text,text)
    return expand_text

def remove_repeated_characters(word):
    pattern = re.compile(r"(\w*)(\w)\2(\w*)")
    substitution_pattern = r"\1\2\3"
    while True:
        if wordnet.synsets(word):
            return word
        new_word = pattern.sub(substitution_pattern,word)
        if new_word != word:
            word = new_word
            continue
        else:
            return new_word

def spelling_checker(word):
    checker = suggest(word)
    return checker[0][0]

def cleanhtml(text):
    cleanr = re.compile('&#[0-9]+;')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

def clean_emoji(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)
    
def normalizer(text):
    text = re.sub(r"http\S+", "", text.lower(), flags=re.MULTILINE) #remove url
    text = re.sub('@[^\s]+','',text) #remove username
    text = clean_emoji(text)
    text = cleanhtml(text)
    expand = expand_contractions(text)
    pattern = re.compile("[{}]".format(re.escape(string.punctuation)))
    filter_char =  filter(None,[pattern.sub('' ,expand)])
    text_filter_char =  " ".join(filter_char)
    tokens = nltk.WhitespaceTokenizer().tokenize(text_filter_char)
    lemmas = [wordnet_lemmatizer.lemmatize(t) for t in tokens]
    stems = [stemmer.stem(t) for t in lemmas]
    filtered_result = list(filter(lambda l: l not in stop_words, stems))
    concate = ' '.join(filtered_result)
    return concate

In [4]:
normalizer('people')

'peopl'

In [5]:
data_physics.sentence = data_physics.sentence.apply(normalizer)
data_race.sentence = data_race.sentence.apply(normalizer)
data_religion.sentence = data_religion.sentence.apply(normalizer)
data_physics.sentence.head()

0    get whi negro alway travel white countri take ...
1                  lmao funni true doe know camara lol
2                                       publicli thank
3                       post pictur khloe alreadi come
4    becom poster child faith partner pleas got caught
Name: sentence, dtype: object

# Split Data Train and Test

In [6]:
from sklearn.cross_validation import train_test_split
SEED = 2000

x_train_physics, x_test_physics, y_train_physics, y_test_physics = train_test_split(data_physics.sentence, data_physics.physics, test_size=.2, random_state=SEED)
x_train_race, x_test_race, y_train_race, y_test_race = train_test_split(data_race.sentence, data_race.race, test_size=.2, random_state=SEED)
x_train_religion, x_test_religion, y_train_religion, y_test_religion = train_test_split(data_religion.sentence, data_religion.religion, test_size=.2, random_state=SEED)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
def labelize_text(text,label):
    result = []
    prefix = label
    for i, t in zip(text.index, text):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result

all_x = pd.concat([x_train_physics, x_test_physics])
all_x_w2v = labelize_text(all_x, 'ALL')

# physics
x_train_physics = labelize_text(x_train_physics, 'TRAIN')
x_test_physics = labelize_text(x_test_physics, 'TEST')

# race
x_train_race = labelize_text(x_train_race, 'TRAIN')
x_test_race = labelize_text(x_test_race, 'TEST')

# religion
x_train_religion = labelize_text(x_train_religion, 'TRAIN')
x_test_religion = labelize_text(x_test_religion, 'TEST')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


# Train Word2Vec

In [8]:
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm
from sklearn import utils
import numpy as np

model_w2v = Word2Vec(size=200, min_count=20)
model_w2v.build_vocab([x.words for x in tqdm(all_x_w2v)])
model_w2v.train([x.words for x in tqdm(all_x_w2v)], total_examples=len(all_x_w2v), epochs=1)

100%|██████████████████████████████████████████████████████████████████████████| 4174/4174 [00:00<00:00, 475514.71it/s]


(8059, 26188)

In [9]:
model_w2v.most_similar('cute')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
C:\Users\User\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('anoth', 0.25935670733451843),
 ('realli', 0.2524394392967224),
 ('would', 0.24629220366477966),
 ('hate', 0.244271919131279),
 ('girl', 0.23824137449264526),
 ('child', 0.22931347787380219),
 ('nice', 0.2237091064453125),
 ('dumb', 0.22180451452732086),
 ('littl', 0.22179366648197174),
 ('pretti', 0.21975255012512207)]

# Build Document Vector using Average Word Vector With TF-IDF

In [10]:
from sklearn.preprocessing import scale
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer=lambda x: x)
matrix = vectorizer.fit_transform([x.words for x in all_x_w2v])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

def build_Word_Vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: 
            
            continue
    if count != 0:
        vec /= count
    return vec

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
# physics
train_vecs_physics = np.concatenate([build_Word_Vector(z, 200) for z in tqdm(map(lambda x: x.words, x_train_physics))])
test_vecs_physics = np.concatenate([build_Word_Vector(z, 200) for z in tqdm(map(lambda x: x.words, x_test_physics))])

# race
train_vecs_race = np.concatenate([build_Word_Vector(z, 200) for z in tqdm(map(lambda x: x.words, x_train_race))])
test_vecs_race = np.concatenate([build_Word_Vector(z, 200) for z in tqdm(map(lambda x: x.words, x_test_race))])

# religion
train_vecs_religion = np.concatenate([build_Word_Vector(z, 200) for z in tqdm(map(lambda x: x.words, x_train_religion))])
test_vecs_religion = np.concatenate([build_Word_Vector(z, 200) for z in tqdm(map(lambda x: x.words, x_test_religion))])

0it [00:00, ?it/s]C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
3339it [00:00, 7624.17it/s]
835it [00:00, 8386.48it/s]
1040it [00:00, 9270.56it/s]
260it [00:00, 6896.82it/s]
688it [00:00, 5736.10it/s]
173it [00:00, 6658.91it/s]


# Bi-LSTM Aspect Detection Model for Physics

In [12]:
batch_size = 1
num_epochs = 100
hidden_size = 10
timesteps = 1
num_class = 1
data_dim = len(train_vecs_physics[0])
num_data = len(train_vecs_physics)
num_data_test = len(test_vecs_physics)

train_vecs_physics = train_vecs_physics.reshape((num_data, timesteps, data_dim))
y_train_physics = y_train_physics.reshape((num_data, num_class))
test_vecs_physics = test_vecs_physics.reshape((num_data_test, timesteps, data_dim))
y_test_physics = y_test_physics.reshape((num_data_test, num_class))

model_ad_physics = Sequential()
model_ad_physics.add(Bidirectional(LSTM(hidden_size, input_shape=(timesteps, data_dim)), merge_mode='concat'))
model_ad_physics.add(Dropout(0.5))
model_ad_physics.add(Dense(1, activation='sigmoid'))
model_ad_physics.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_ad_physics.fit(train_vecs_physics, y_train_physics, epochs=num_epochs, validation_data=[test_vecs_physics, y_test_physics])

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  # This is added back by InteractiveShellApp.init_path()
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  del sys.path[0]


Train on 3339 samples, validate on 835 samples
Epoch 1/100
3339/3339 [==============================] - ETA: 4:20 - loss: 0.6931 - acc: 0.593 - ETA: 30s - loss: 0.6928 - acc: 0.535 - ETA: 13s - loss: 0.6928 - acc: 0.52 - ETA: 8s - loss: 0.6926 - acc: 0.5240 - ETA: 4s - loss: 0.6924 - acc: 0.524 - ETA: 3s - loss: 0.6922 - acc: 0.525 - ETA: 2s - loss: 0.6923 - acc: 0.518 - ETA: 1s - loss: 0.6923 - acc: 0.515 - ETA: 1s - loss: 0.6921 - acc: 0.516 - ETA: 0s - loss: 0.6920 - acc: 0.526 - ETA: 0s - loss: 0.6918 - acc: 0.538 - ETA: 0s - loss: 0.6917 - acc: 0.537 - 4s 1ms/step - loss: 0.6916 - acc: 0.5370 - val_loss: 0.6895 - val_acc: 0.6539
Epoch 2/100
3339/3339 [==============================] - ETA: 0s - loss: 0.6875 - acc: 0.750 - ETA: 0s - loss: 0.6885 - acc: 0.664 - ETA: 0s - loss: 0.6889 - acc: 0.639 - ETA: 0s - loss: 0.6891 - acc: 0.613 - ETA: 0s - loss: 0.6890 - acc: 0.608 - ETA: 0s - loss: 0.6888 - acc: 0.615 - ETA: 0s - loss: 0.6884 - acc: 0.631 - ETA: 0s - loss: 0.6883 - acc: 0.641

Epoch 12/100
3339/3339 [==============================] - ETA: 0s - loss: 0.4663 - acc: 0.812 - ETA: 0s - loss: 0.4892 - acc: 0.784 - ETA: 0s - loss: 0.4924 - acc: 0.791 - ETA: 0s - loss: 0.4742 - acc: 0.807 - ETA: 0s - loss: 0.4798 - acc: 0.808 - ETA: 0s - loss: 0.4756 - acc: 0.812 - ETA: 0s - loss: 0.4743 - acc: 0.816 - ETA: 0s - loss: 0.4731 - acc: 0.817 - ETA: 0s - loss: 0.4791 - acc: 0.813 - ETA: 0s - loss: 0.4804 - acc: 0.814 - ETA: 0s - loss: 0.4791 - acc: 0.815 - ETA: 0s - loss: 0.4804 - acc: 0.812 - ETA: 0s - loss: 0.4788 - acc: 0.811 - ETA: 0s - loss: 0.4779 - acc: 0.810 - ETA: 0s - loss: 0.4806 - acc: 0.808 - ETA: 0s - loss: 0.4807 - acc: 0.805 - ETA: 0s - loss: 0.4820 - acc: 0.804 - 1s 287us/step - loss: 0.4809 - acc: 0.8056 - val_loss: 0.4552 - val_acc: 0.8048
Epoch 13/100
3339/3339 [==============================] - ETA: 1s - loss: 0.6229 - acc: 0.781 - ETA: 0s - loss: 0.4770 - acc: 0.808 - ETA: 0s - loss: 0.4806 - acc: 0.808 - ETA: 0s - loss: 0.4749 - acc: 0.815 - ETA: 0

3339/3339 [==============================] - ETA: 0s - loss: 0.3016 - acc: 0.906 - ETA: 0s - loss: 0.3465 - acc: 0.888 - ETA: 0s - loss: 0.3860 - acc: 0.868 - ETA: 0s - loss: 0.3908 - acc: 0.865 - ETA: 0s - loss: 0.3631 - acc: 0.869 - ETA: 0s - loss: 0.3723 - acc: 0.852 - ETA: 0s - loss: 0.3727 - acc: 0.857 - ETA: 0s - loss: 0.3787 - acc: 0.850 - ETA: 0s - loss: 0.3928 - acc: 0.843 - ETA: 0s - loss: 0.3896 - acc: 0.844 - ETA: 0s - loss: 0.3878 - acc: 0.845 - ETA: 0s - loss: 0.3925 - acc: 0.841 - ETA: 0s - loss: 0.3970 - acc: 0.839 - ETA: 0s - loss: 0.3991 - acc: 0.840 - ETA: 0s - loss: 0.3999 - acc: 0.841 - ETA: 0s - loss: 0.3991 - acc: 0.841 - 1s 272us/step - loss: 0.3985 - acc: 0.8410 - val_loss: 0.3875 - val_acc: 0.8455
Epoch 33/100
3339/3339 [==============================] - ETA: 0s - loss: 0.4814 - acc: 0.718 - ETA: 0s - loss: 0.3618 - acc: 0.843 - ETA: 0s - loss: 0.3661 - acc: 0.834 - ETA: 0s - loss: 0.3749 - acc: 0.830 - ETA: 0s - loss: 0.3798 - acc: 0.832 - ETA: 0s - loss: 0.3

Epoch 43/100
3339/3339 [==============================] - ETA: 0s - loss: 0.5672 - acc: 0.750 - ETA: 0s - loss: 0.4499 - acc: 0.816 - ETA: 0s - loss: 0.4265 - acc: 0.822 - ETA: 0s - loss: 0.3944 - acc: 0.839 - ETA: 0s - loss: 0.3861 - acc: 0.844 - ETA: 0s - loss: 0.3746 - acc: 0.847 - ETA: 0s - loss: 0.3843 - acc: 0.846 - ETA: 0s - loss: 0.3852 - acc: 0.845 - ETA: 0s - loss: 0.3897 - acc: 0.841 - ETA: 0s - loss: 0.3930 - acc: 0.840 - ETA: 0s - loss: 0.3997 - acc: 0.836 - ETA: 0s - loss: 0.3935 - acc: 0.837 - ETA: 0s - loss: 0.3942 - acc: 0.840 - ETA: 0s - loss: 0.3892 - acc: 0.841 - ETA: 0s - loss: 0.3939 - acc: 0.843 - ETA: 0s - loss: 0.3906 - acc: 0.845 - ETA: 0s - loss: 0.3899 - acc: 0.845 - 1s 280us/step - loss: 0.3894 - acc: 0.8464 - val_loss: 0.3742 - val_acc: 0.8467
Epoch 44/100
3339/3339 [==============================] - ETA: 0s - loss: 0.3687 - acc: 0.843 - ETA: 0s - loss: 0.3906 - acc: 0.878 - ETA: 0s - loss: 0.3840 - acc: 0.872 - ETA: 0s - loss: 0.3997 - acc: 0.851 - ETA: 0

Epoch 64/100
3339/3339 [==============================] - ETA: 1s - loss: 0.2714 - acc: 0.906 - ETA: 0s - loss: 0.4671 - acc: 0.852 - ETA: 0s - loss: 0.3989 - acc: 0.872 - ETA: 0s - loss: 0.3852 - acc: 0.870 - ETA: 0s - loss: 0.3972 - acc: 0.858 - ETA: 0s - loss: 0.4072 - acc: 0.852 - ETA: 0s - loss: 0.3983 - acc: 0.850 - ETA: 0s - loss: 0.3941 - acc: 0.849 - ETA: 0s - loss: 0.3875 - acc: 0.850 - ETA: 0s - loss: 0.3861 - acc: 0.851 - ETA: 0s - loss: 0.3879 - acc: 0.846 - ETA: 0s - loss: 0.3815 - acc: 0.848 - ETA: 0s - loss: 0.3854 - acc: 0.847 - ETA: 0s - loss: 0.3846 - acc: 0.848 - ETA: 0s - loss: 0.3836 - acc: 0.848 - ETA: 0s - loss: 0.3806 - acc: 0.850 - ETA: 0s - loss: 0.3742 - acc: 0.852 - 1s 278us/step - loss: 0.3747 - acc: 0.8524 - val_loss: 0.3633 - val_acc: 0.8455
Epoch 65/100
3339/3339 [==============================] - ETA: 0s - loss: 0.2379 - acc: 0.968 - ETA: 0s - loss: 0.3519 - acc: 0.875 - ETA: 0s - loss: 0.3567 - acc: 0.862 - ETA: 0s - loss: 0.3690 - acc: 0.853 - ETA: 0

3339/3339 [==============================] - ETA: 0s - loss: 0.2769 - acc: 0.875 - ETA: 0s - loss: 0.3969 - acc: 0.832 - ETA: 0s - loss: 0.3570 - acc: 0.856 - ETA: 0s - loss: 0.3548 - acc: 0.858 - ETA: 0s - loss: 0.3627 - acc: 0.854 - ETA: 0s - loss: 0.3654 - acc: 0.855 - ETA: 0s - loss: 0.3697 - acc: 0.852 - ETA: 0s - loss: 0.3659 - acc: 0.855 - ETA: 0s - loss: 0.3661 - acc: 0.856 - ETA: 0s - loss: 0.3717 - acc: 0.855 - ETA: 0s - loss: 0.3785 - acc: 0.850 - ETA: 0s - loss: 0.3766 - acc: 0.852 - ETA: 0s - loss: 0.3711 - acc: 0.855 - ETA: 0s - loss: 0.3679 - acc: 0.856 - ETA: 0s - loss: 0.3637 - acc: 0.858 - ETA: 0s - loss: 0.3627 - acc: 0.859 - ETA: 0s - loss: 0.3618 - acc: 0.859 - 1s 288us/step - loss: 0.3623 - acc: 0.8589 - val_loss: 0.3572 - val_acc: 0.8575
Epoch 86/100
3339/3339 [==============================] - ETA: 1s - loss: 0.3496 - acc: 0.875 - ETA: 0s - loss: 0.3893 - acc: 0.835 - ETA: 0s - loss: 0.3749 - acc: 0.845 - ETA: 0s - loss: 0.3926 - acc: 0.840 - ETA: 0s - loss: 0.3

In [13]:
from sklearn.metrics import accuracy_score, classification_report

prediction = {}
prediction['ad_physics'] = model_ad_physics.predict(test_vecs_physics)

for i in range(len(prediction['ad_physics'])):
    prediction['ad_physics'][i][0] = round(prediction['ad_physics'][i][0])

accuracy = {}
accuracy['ad_physics'] = accuracy_score(y_test_physics, prediction['ad_physics'])
print("Accuracy: ", accuracy['ad_physics'], "\n")
print(classification_report(y_test_physics, prediction['ad_physics'], labels = [0, 1]))

Accuracy:  0.8562874251497006 

             precision    recall  f1-score   support

          0       0.84      0.89      0.87       431
          1       0.88      0.82      0.85       404

avg / total       0.86      0.86      0.86       835



# Bi-LSTM Aspect Detection Model for Race

In [14]:
data_dim = len(train_vecs_race[0])
num_data = len(train_vecs_race)
num_data_test = len(test_vecs_race)

train_vecs_race = train_vecs_race.reshape((num_data, timesteps, data_dim))
y_train_race = y_train_race.reshape((num_data, num_class))
test_vecs_race = test_vecs_race.reshape((num_data_test, timesteps, data_dim))
y_test_race = y_test_race.reshape((num_data_test, num_class))

model_ad_race = Sequential()
model_ad_race.add(Bidirectional(LSTM(hidden_size, input_shape=(timesteps, data_dim)), merge_mode='concat'))
model_ad_race.add(Dropout(0.5))
model_ad_race.add(Dense(1, activation='sigmoid'))
model_ad_race.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_ad_race.fit(train_vecs_race, y_train_race, epochs=num_epochs, validation_data=[test_vecs_race, y_test_race])

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


Train on 1040 samples, validate on 260 samples
Epoch 1/100
1040/1040 [==============================] - ETA: 1:29 - loss: 0.6931 - acc: 0.593 - ETA: 10s - loss: 0.6924 - acc: 0.611 - ETA: 5s - loss: 0.6921 - acc: 0.6042 - ETA: 2s - loss: 0.6909 - acc: 0.618 - ETA: 0s - loss: 0.6901 - acc: 0.623 - ETA: 0s - loss: 0.6889 - acc: 0.630 - 3s 3ms/step - loss: 0.6886 - acc: 0.6327 - val_loss: 0.6846 - val_acc: 0.6346
Epoch 2/100
1040/1040 [==============================] - ETA: 0s - loss: 0.6847 - acc: 0.625 - ETA: 0s - loss: 0.6816 - acc: 0.656 - ETA: 0s - loss: 0.6824 - acc: 0.640 - ETA: 0s - loss: 0.6805 - acc: 0.651 - ETA: 0s - loss: 0.6819 - acc: 0.633 - ETA: 0s - loss: 0.6808 - acc: 0.634 - 0s 307us/step - loss: 0.6803 - acc: 0.6385 - val_loss: 0.6765 - val_acc: 0.6346
Epoch 3/100
1040/1040 [==============================] - ETA: 0s - loss: 0.6856 - acc: 0.562 - ETA: 0s - loss: 0.6759 - acc: 0.632 - ETA: 0s - loss: 0.6722 - acc: 0.645 - ETA: 0s - loss: 0.6706 - acc: 0.651 - ETA: 0s - lo

1040/1040 [==============================] - ETA: 0s - loss: 0.3230 - acc: 0.812 - ETA: 0s - loss: 0.3709 - acc: 0.840 - ETA: 0s - loss: 0.3953 - acc: 0.840 - ETA: 0s - loss: 0.3895 - acc: 0.846 - 0s 194us/step - loss: 0.3795 - acc: 0.8500 - val_loss: 0.4215 - val_acc: 0.8192
Epoch 55/100
1040/1040 [==============================] - ETA: 0s - loss: 0.3485 - acc: 0.875 - ETA: 0s - loss: 0.4153 - acc: 0.828 - ETA: 0s - loss: 0.3727 - acc: 0.847 - ETA: 0s - loss: 0.3702 - acc: 0.849 - 0s 214us/step - loss: 0.3788 - acc: 0.8433 - val_loss: 0.4215 - val_acc: 0.8154
Epoch 56/100
1040/1040 [==============================] - ETA: 0s - loss: 0.3309 - acc: 0.875 - ETA: 0s - loss: 0.3532 - acc: 0.849 - ETA: 0s - loss: 0.3682 - acc: 0.856 - ETA: 0s - loss: 0.3700 - acc: 0.850 - 0s 205us/step - loss: 0.3704 - acc: 0.8442 - val_loss: 0.4205 - val_acc: 0.8192
Epoch 57/100
1040/1040 [==============================] - ETA: 0s - loss: 0.2507 - acc: 0.906 - ETA: 0s - loss: 0.4190 - acc: 0.858 - ETA: 0s -

Epoch 82/100
1040/1040 [==============================] - ETA: 0s - loss: 0.3411 - acc: 0.812 - ETA: 0s - loss: 0.3296 - acc: 0.871 - ETA: 0s - loss: 0.3391 - acc: 0.864 - ETA: 0s - loss: 0.3362 - acc: 0.869 - 0s 225us/step - loss: 0.3415 - acc: 0.8692 - val_loss: 0.4079 - val_acc: 0.8231
Epoch 83/100
1040/1040 [==============================] - ETA: 0s - loss: 0.4004 - acc: 0.812 - ETA: 0s - loss: 0.3171 - acc: 0.863 - ETA: 0s - loss: 0.3291 - acc: 0.871 - ETA: 0s - loss: 0.3315 - acc: 0.869 - 0s 231us/step - loss: 0.3450 - acc: 0.8606 - val_loss: 0.4083 - val_acc: 0.8192
Epoch 84/100
1040/1040 [==============================] - ETA: 0s - loss: 0.2498 - acc: 0.906 - ETA: 0s - loss: 0.3089 - acc: 0.888 - ETA: 0s - loss: 0.3101 - acc: 0.882 - ETA: 0s - loss: 0.3142 - acc: 0.885 - ETA: 0s - loss: 0.3425 - acc: 0.871 - 0s 255us/step - loss: 0.3443 - acc: 0.8683 - val_loss: 0.4076 - val_acc: 0.8231
Epoch 85/100
1040/1040 [==============================] - ETA: 0s - loss: 0.3001 - acc: 0.84

In [15]:
prediction['ad_race'] = model_ad_race.predict(test_vecs_race)

for i in range(len(prediction['ad_race'])):
    prediction['ad_race'][i][0] = round(prediction['ad_race'][i][0])

accuracy['ad_race'] = accuracy_score(y_test_race, prediction['ad_race'])
print("Accuracy: ", accuracy['ad_race'], "\n")
print(classification_report(y_test_race, prediction['ad_race'], labels = [0, 1]))

Accuracy:  0.8269230769230769 

             precision    recall  f1-score   support

          0       0.82      0.93      0.87       165
          1       0.84      0.65      0.73        95

avg / total       0.83      0.83      0.82       260



# Bi-LSTM Aspect Detection Model for Religion

In [16]:
data_dim = len(train_vecs_religion[0])
num_data = len(train_vecs_religion)
num_data_test = len(test_vecs_religion)

train_vecs_religion = train_vecs_religion.reshape((num_data, timesteps, data_dim))
y_train_religion = y_train_religion.reshape((num_data, num_class))
test_vecs_religion = test_vecs_religion.reshape((num_data_test, timesteps, data_dim))
y_test_religion = y_test_religion.reshape((num_data_test, num_class))

model_ad_religion = Sequential()
model_ad_religion.add(Bidirectional(LSTM(hidden_size, input_shape=(timesteps, data_dim)), merge_mode='concat'))
model_ad_religion.add(Dropout(0.5))
model_ad_religion.add(Dense(1, activation='sigmoid'))
model_ad_religion.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_ad_religion.fit(train_vecs_religion, y_train_religion, epochs=num_epochs, validation_data=[test_vecs_religion, y_test_religion])

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


Train on 688 samples, validate on 173 samples
Epoch 1/100
688/688 [==============================] - ETA: 1:01 - loss: 0.6928 - acc: 0.593 - ETA: 4s - loss: 0.6923 - acc: 0.5903  - ETA: 0s - loss: 0.6912 - acc: 0.601 - 4s 5ms/step - loss: 0.6913 - acc: 0.5974 - val_loss: 0.6894 - val_acc: 0.5954
Epoch 2/100
688/688 [==============================] - ETA: 0s - loss: 0.6865 - acc: 0.656 - ETA: 0s - loss: 0.6888 - acc: 0.590 - ETA: 0s - loss: 0.6885 - acc: 0.586 - 0s 221us/step - loss: 0.6875 - acc: 0.6017 - val_loss: 0.6863 - val_acc: 0.5954
Epoch 3/100
688/688 [==============================] - ETA: 0s - loss: 0.6912 - acc: 0.500 - ETA: 0s - loss: 0.6840 - acc: 0.609 - ETA: 0s - loss: 0.6846 - acc: 0.598 - 0s 228us/step - loss: 0.6842 - acc: 0.6017 - val_loss: 0.6830 - val_acc: 0.5954
Epoch 4/100
688/688 [==============================] - ETA: 0s - loss: 0.6692 - acc: 0.718 - ETA: 0s - loss: 0.6771 - acc: 0.633 - ETA: 0s - loss: 0.6796 - acc: 0.609 - 0s 225us/step - loss: 0.6808 - acc: 

Epoch 34/100
688/688 [==============================] - ETA: 0s - loss: 0.5584 - acc: 0.750 - ETA: 0s - loss: 0.5869 - acc: 0.684 - 0s 192us/step - loss: 0.5963 - acc: 0.6875 - val_loss: 0.5841 - val_acc: 0.7052
Epoch 35/100
688/688 [==============================] - ETA: 0s - loss: 0.6079 - acc: 0.656 - ETA: 0s - loss: 0.5800 - acc: 0.735 - ETA: 0s - loss: 0.5901 - acc: 0.706 - 0s 229us/step - loss: 0.5901 - acc: 0.7049 - val_loss: 0.5772 - val_acc: 0.7399
Epoch 36/100
688/688 [==============================] - ETA: 0s - loss: 0.5398 - acc: 0.750 - ETA: 0s - loss: 0.5990 - acc: 0.696 - ETA: 0s - loss: 0.5885 - acc: 0.713 - 0s 213us/step - loss: 0.5838 - acc: 0.7209 - val_loss: 0.5700 - val_acc: 0.7861
Epoch 37/100
688/688 [==============================] - ETA: 0s - loss: 0.5591 - acc: 0.843 - ETA: 0s - loss: 0.5835 - acc: 0.730 - ETA: 0s - loss: 0.5861 - acc: 0.722 - 0s 221us/step - loss: 0.5823 - acc: 0.7267 - val_loss: 0.5621 - val_acc: 0.7861
Epoch 38/100
688/688 [================

Epoch 100/100
688/688 [==============================] - ETA: 0s - loss: 0.5213 - acc: 0.781 - ETA: 0s - loss: 0.3707 - acc: 0.843 - ETA: 0s - loss: 0.3733 - acc: 0.845 - 0s 222us/step - loss: 0.3823 - acc: 0.8445 - val_loss: 0.3418 - val_acc: 0.8613


In [17]:
prediction['ad_religion'] = model_ad_religion.predict(test_vecs_religion)

for i in range(len(prediction['ad_religion'])):
    prediction['ad_religion'][i][0] = round(prediction['ad_religion'][i][0])

accuracy['ad_religion'] = accuracy_score(y_test_religion, prediction['ad_religion'])
print("Accuracy: ", accuracy['ad_religion'], "\n")
print(classification_report(y_test_religion, prediction['ad_religion'], labels = [0, 1]))

Accuracy:  0.861271676300578 

             precision    recall  f1-score   support

          0       0.86      0.91      0.89       103
          1       0.86      0.79      0.82        70

avg / total       0.86      0.86      0.86       173



# Save Model

In [18]:
import pickle

pickle.dump(model_ad_physics, open('model_ad_physics.sav', 'wb'))
pickle.dump(model_ad_race, open('model_ad_race.sav', 'wb'))
pickle.dump(model_ad_religion, open('model_ad_religion.sav', 'wb'))

# Load Model

In [19]:
model_ad_physics = pickle.load(open('model_ad_physics.sav', 'rb'))
model_ad_race = pickle.load(open('model_ad_race.sav', 'rb'))
model_ad_religion = pickle.load(open('model_ad_religion.sav', 'rb'))

# Aspect Detection Model for Predict New Data 

In [30]:
# 1:exist, 0:not given

def predict_aspect(text):
    text = normalizer(text)
    tokens = nltk.WhitespaceTokenizer().tokenize(text)
    vecs = build_Word_Vector(tokens, 200)
    vecs = vecs.reshape((1, 1, 200))
    aspect = {}
    aspect['physics'] = int(round(model_ad_physics.predict(vecs)[0][0]))
    aspect['race'] = int(round(model_ad_race.predict(vecs)[0][0]))
    aspect['religion'] = int(round(model_ad_religion.predict(vecs)[0][0]))
    return aspect

In [33]:
predict_aspect("nigga bitch")

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


{'physics': 1, 'race': 1, 'religion': 0}